credit : https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#define-the-network

## Neural Networks
Neural networks can be constructed using the torch.nn package
An nn.Module contains layers, and a method forward(input) that returns the output

https://pytorch.org/docs/stable/nn.html#linear

## Define the network

Inputs : 8x1x32x32 (nSamples x nChannels x Height x Width)

8 is mini batch's size, if input is RGB image, then nChannels is 3 

Output : 8x10 (nSamples x nClasses)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# inherit from nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y=Wx+b
        # define fully connect layer
        self.fc1 = nn.Linear(32*32,120)     # 32*32 mapping to 120
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU(inplace=True)   
        # inplace-can optionally do the operation in-place.
        # inplace true means that it will modify the input directly, without allocating any additional output. It can sometimes slightly decrease the memory usage, but may not always be a valid operation (because the original input is destroyed). However, if you don't see an error, it means that your use case is valid
        
    def forward(self, x):
        # View returns a new tensor with the same data but of a diffent size(resizing)
        x = x.view(-1,32*32)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)
        

Net(
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU(inplace)
)


You just have to define the forward function, and the backward function( where gradients are computed) is automatically defined for you using autograd.

### You can use any of the Tensor operations in the forward function.

In [2]:
class AnyOperations(nn.Module):
    def __init__(self):
        super(AnyOperations, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(32*32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = x.view(-1,32*32)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        # noise = torch.Tensor(3).normal_(0,1)
        for i in range(10):
            if torch.rand(1).item()>0.5:
                x = x*x
            else:
                x = x+x
        return x

net2 = AnyOperations()
print(net2)

AnyOperations(
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU(inplace)
)


The learnable parameters of a model are returned by net.parameters()

In [3]:
params = list(net.parameters())
print(params)

[Parameter containing:
tensor([[ 0.0089, -0.0263, -0.0170,  ...,  0.0221,  0.0092,  0.0173],
        [-0.0006, -0.0183, -0.0070,  ..., -0.0012, -0.0017,  0.0051],
        [ 0.0165,  0.0152, -0.0228,  ...,  0.0261, -0.0013, -0.0052],
        ...,
        [ 0.0291, -0.0128, -0.0271,  ..., -0.0169, -0.0161, -0.0255],
        [-0.0008,  0.0053, -0.0067,  ...,  0.0211,  0.0126,  0.0132],
        [-0.0150,  0.0104, -0.0259,  ...,  0.0112,  0.0226,  0.0241]],
       requires_grad=True), Parameter containing:
tensor([-0.0072, -0.0258,  0.0082,  0.0125, -0.0073, -0.0235,  0.0269, -0.0132,
         0.0183, -0.0081, -0.0137, -0.0259,  0.0149,  0.0206,  0.0076, -0.0071,
        -0.0103, -0.0174, -0.0183, -0.0170, -0.0154,  0.0247, -0.0075,  0.0029,
        -0.0103,  0.0238,  0.0081,  0.0127, -0.0021, -0.0017, -0.0228, -0.0151,
         0.0019, -0.0297,  0.0112, -0.0259,  0.0118,  0.0171, -0.0005, -0.0206,
        -0.0079, -0.0256,  0.0151, -0.0158,  0.0027, -0.0214, -0.0156, -0.0029,
        -0.00

In [4]:
print(len(params))

6


In [5]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0441, -0.0706, -0.0051, -0.0244, -0.0264,  0.0856,  0.0977, -0.0560,
          0.0734, -0.0126]], grad_fn=<ThAddmmBackward>)


## Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package. A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.

For example:

In [6]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9547, grad_fn=<MseLossBackward>)


## Backprop
To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call loss.backward(), and have a look at fc1's bias gradients before and after the backward

In [7]:
net.zero_grad()    # zeroes the gradient buffers of all parameters

print('fc1.bias.grad before backward')
print(net.fc1.bias.grad)
print('\n\n')

# gradient 계산
loss.backward()

print('fc1.bias.grad after backward')
print(net.fc1.bias.grad)

fc1.bias.grad before backward
None



fc1.bias.grad after backward
tensor([ 0.0009,  0.0000,  0.0027,  0.0040,  0.0000,  0.0000, -0.0200,  0.0000,
         0.0000,  0.0000,  0.0040,  0.0000,  0.0000,  0.0165,  0.0091,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0112, -0.0061,  0.0000,  0.0165, -0.0245,
         0.0000,  0.0053,  0.0164,  0.0102,  0.0000, -0.0153,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0104, -0.0124,  0.0000, -0.0113, -0.0116,
         0.0068,  0.0010, -0.0022,  0.0000,  0.0000,  0.0002,  0.0000,  0.0000,
        -0.0084,  0.0114, -0.0122,  0.0000,  0.0000, -0.0033,  0.0092,  0.0000,
        -0.0055,  0.0000,  0.0078,  0.0000,  0.0000, -0.0118,  0.0000,  0.0095,
        -0.0083,  0.0098,  0.0045,  0.0000,  0.0017,  0.0000,  0.0054,  0.0000,
         0.0085, -0.0091, -0.0197,  0.0000,  0.0000,  0.0000, -0.0024,  0.0000,
         0.0000,  0.0177, -0.0161,  0.0000,  0.0043,  0.0086,  0.0024,  0.0120,
         0.0000, -0.0007, -0.0124, -0.0062,  0.0058, 

## Update the weights
The simplest update rule used in practice is the Stochastic Gradient Descent(SGD):
weight = weight - learning_rate * gradient
We can implement this using simple python code:

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers (prevent gradient accumulated)
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Do update

Observe how gradient buffers had to be manually set to zero using optimizer.zero_grad(). This is because gradients are accumulated as explained in Backprop section